In [1]:
import pandas as pd
import numpy as np
import xmltodict

In [2]:
import urllib2
import datetime

In [3]:
def read_data(filenames):
    arrays = [0,0,0]
    for i in xrange(len(filenames)):
        arrays[i] = pd.read_csv(filenames[i], parse_dates=['timestamp'])
    return arrays

In [4]:
def my_transport(data):
    data["metro_km_walk"]=data["metro_km_walk"].fillna(data["metro_km_avto"])
    data["metro_min_walk"]=data["metro_min_walk"].fillna(data["metro_km_avto"]*12)
    data["railroad_station_walk_km"]=data["railroad_station_walk_km"].fillna(data["railroad_station_avto_km"])
    data["railroad_station_walk_min"]=data["railroad_station_walk_min"].fillna(data["railroad_station_avto_km"]*12)
    data["ID_railroad_station_walk"]=data["ID_railroad_station_walk"].fillna(data["ID_railroad_station_avto"])
    return data

In [5]:
from sklearn.preprocessing import Imputer
def my_cafe(data):
    st0=['cafe_sum_5000_min_price_avg','cafe_sum_5000_max_price_avg','cafe_avg_price_5000']
    l=['5000','3000','2000','1500','1000','500']
    medImputer=Imputer(strategy='median')
    data[st0]=medImputer.fit_transform(data[st0])
    for i in xrange(len(l)-1):
        st1=[st.replace('5000',str(l[i+1])) for st in st0]
        st2=[st.replace('5000',str(l[i])) for st in st0]
        for k in xrange(len(st1)):
            data[st1[k]]=data[st1[k]].fillna(data[st2[k]])
    return data        

In [6]:
def er10(data):
    err = (data.full_sq>data.life_sq*10)&(data.life_sq>2)
    data.ix[err,'full_sq'] = data.ix[err,'full_sq']/10
    return data

In [7]:
def my_impute(data):
    data["floor"]=data["floor"].fillna(data["max_floor"])
    data["life_sq"]=(data["life_sq"]/data["full_sq"]).replace(np.inf,1)
    data["kitch_sq"]=(data["kitch_sq"]/data["full_sq"]).replace(np.inf,1)
    data["max_floor"]=(data["max_floor"]/data["floor"]).replace(np.inf,1)
    data["num_room"]=(data["num_room"]/data["full_sq"]).replace(np.inf,1)
    data.replace(np.inf, np.nan)
    data['green_part_2000']=data['green_part_2000'].fillna(data['green_part_1500'])
    data['prom_part_5000']=data['prom_part_5000'].fillna(data['prom_part_2000'])
    #data=data.apply(yes_1)
    medImputer=Imputer(strategy='median')
    mfImputer=Imputer(strategy='most_frequent')
    data[["material","build_year","state","floor"]]=mfImputer.fit_transform(data[["material","build_year","state","floor"]])
    data[["life_sq","max_floor","num_room","kitch_sq"]]=medImputer.fit_transform(data[["life_sq","max_floor","num_room","kitch_sq"]])
    data["build_year"]=2017-data["build_year"]
    return data   

In [8]:
def new_features(train):
    
    #train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
    #train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)
    train.apartment_name=train.sub_area + train['metro_km_avto'].astype(str)
    #train['room_size'] = 1 / train['num_room'].astype(float)
    
    return train

In [9]:
def labels(x_train):   
    for c in x_train.columns:
        if x_train[c].dtype == 'object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(x_train[c].values))
            x_train[c] = lbl.transform(list(x_train[c].values))
            #print c
            #x_train.drop(c,axis=1,inplace=True)    
    return x_train   

In [10]:
def add_dates(train):
    
    # Add month-year
    month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
    month_year_cnt_map = month_year.value_counts().to_dict()
    train['month_year_cnt'] = month_year.map(month_year_cnt_map)
    
    # Add week-year count
    week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
    week_year_cnt_map = week_year.value_counts().to_dict()
    train['week_year_cnt'] = week_year.map(week_year_cnt_map)
    
    # Add month and day-of-week
    train['month'] = train.timestamp.dt.month
    train['dow'] = train.timestamp.dt.dayofweek
    
    return train
    

In [11]:
def printna(data):
    k=len(data)-data.count()
    return k[k>0]

In [12]:
def clean_sq(train):
    bad_index = train[(train.life_sq > train.full_sq) | 
                      (train.life_sq < 5)].index
    train.ix[bad_index, "life_sq"] = np.NaN

    bad_index = train[(train.full_sq > 210) & (train.life_sq / train.full_sq < 0.3) | 
                      (train.full_sq < 5)].index
    train.ix[bad_index, "full_sq"] = np.NaN
    
    bad_index = train[(train.kitch_sq >= train.life_sq) |
                      (train.kitch_sq == 0) | 
                      (train.kitch_sq == 1)].index
    train.ix[bad_index, "kitch_sq"] = np.NaN
    
    bad_index = train[train.life_sq > 300].index
    train.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
    
    bad_index = train[train.build_year < 1500].index
    train.ix[bad_index, "build_year"] = np.NaN
    
    bad_index = train[train.num_room == 0].index 
    train.ix[bad_index, "num_room"] = np.NaN
    
    bad_index = train[train.max_floor == 0].index
    train.ix[bad_index, "max_floor"] = np.NaN
    
    bad_index = train[train.floor > train.max_floor].index
    train.ix[bad_index, "max_floor"] = np.NaN    
    
    return(train)    

In [13]:
def bad_address(data):
    data=data.set_index('id')
    fx = pd.read_excel('./data/BAD_ADDRESS_FIX.xlsx').drop_duplicates('id').set_index('id')
    data.update(fx)
    print('Fix: ', data.index.intersection(fx.index).shape[0])
    return data.reset_index()

In [14]:
def month_median(macro):
    macro["year"]  = macro["timestamp"].dt.year
    macro["month"] = macro["timestamp"].dt.month
    macro["yearmonth"] = 100*macro.year + macro.month
    macmeds = macro.groupby("yearmonth").median()
    return macmeds

In [15]:
# Parameters
micro_humility_factor = 1     #    range from 0 (complete humility) to 1 (no humility)
macro_humility_factor = 0.96

In [16]:
filenames = ["./data/macro.csv","./data/train.csv/train.csv","./data/test.csv/test.csv"]
[macro, train, test] = read_data(filenames)
[macro, train] = map(month_median,[macro, train])
id_test = test.id
df = macro.join(train, lsuffix='_left')

In [17]:
import numpy.matlib as ml
import seaborn as sns 
def almonZmatrix(X, maxlag, maxdeg):
    """
    Creates the Z matrix corresponding to vector X.
    """
    n = len(X)
    Z = ml.zeros((len(X)-maxlag, maxdeg+1))
    for t in range(maxlag,  n):
       #Solve for Z[t][0].
       Z[t-maxlag,0] = sum([X[t-lag] for lag in range(maxlag+1)])
       for j in range(1, maxdeg+1):
             s = 0.0
             for i in range(1, maxlag+1):       
                s += (i)**j * X[t-i]
             Z[t-maxlag,j] = s
    return Z

In [18]:
# Prepare data for macro model
import statsmodels.api as sm
y = df.price_doc.div(df.cpi).apply(np.log).loc[201108:201506]
lncpi = df.cpi.apply(np.log)
tblags = 5    # Number of lags used on PDL for Trade Balance
mrlags = 5    # Number of lags used on PDL for Mortgage Rate
cplags = 5    # Number of lags used on PDL for CPI
ztb = almonZmatrix(df.balance_trade.loc[201103:201506].as_matrix(), tblags, 1)
zmr = almonZmatrix(df.mortgage_rate.loc[201103:201506].as_matrix(), mrlags, 1)
zcp = almonZmatrix(lncpi.loc[201103:201506].as_matrix(), cplags, 1)
columns = ['tb0', 'tb1', 'mr0', 'mr1', 'cp0', 'cp1']
z = pd.DataFrame( np.concatenate( (ztb, zmr, zcp), axis=1), y.index.values, columns )
X = sm.add_constant( z )

# Fit macro model
eq = sm.OLS(y, X)
fit = eq.fit()

# Predict with macro model
test_cpi = df.cpi.loc[201507:201605]
test_index = test_cpi.index
ztb_test = almonZmatrix(df.balance_trade.loc[201502:201605].as_matrix(), tblags, 1)
zmr_test = almonZmatrix(df.mortgage_rate.loc[201502:201605].as_matrix(), mrlags, 1)
zcp_test = almonZmatrix(lncpi.loc[201502:201605].as_matrix(), cplags, 1)
z_test = pd.DataFrame( np.concatenate( (ztb_test, zmr_test, zcp_test), axis=1), 
                       test_index, columns )
X_test = sm.add_constant( z_test )
pred_lnrp = fit.predict( X_test )
pred_p = np.exp(pred_lnrp) * test_cpi

# Merge with test cases and compute mean for macro prediction
test["timestamp"] = pd.to_datetime(test["timestamp"])
test["year"]  = test["timestamp"].dt.year
test["month"] = test["timestamp"].dt.month
test["yearmonth"] = 100*test.year + test.month
test_ids = test[["yearmonth","id"]]
monthprices = pd.DataFrame({"yearmonth":pred_p.index.values,"monthprice":pred_p.values})
macro_mean = np.exp(test_ids.merge(monthprices, on="yearmonth").monthprice.apply(np.log).mean())
macro_mean

6593788.9177231779

In [19]:
# Naive macro model assumes housing prices will simply follow CPI
naive_pred_lnrp = y.mean()
naive_pred_p = np.exp(naive_pred_lnrp) * test_cpi
monthnaive = pd.DataFrame({"yearmonth":pred_p.index.values, "monthprice":naive_pred_p.values})
macro_naive = np.exp(test_ids.merge(monthnaive, on="yearmonth").monthprice.apply(np.log).mean())
macro_naive

7773440.7512487005

In [20]:
# Combine naive and substantive macro models
macro_mean = macro_naive * (macro_mean/macro_naive) ** macro_humility_factor
macro_mean

6637341.6089008758

In [21]:
filenames = ["./data/macro.csv","./data/train.csv/train.csv","./data/test.csv/test.csv"]
[macro, train, test] = read_data(filenames)
train, test = map(bad_address, [train, test])
train=er10(er10(train))
train, test = map(my_transport, [train, test])
train, test = map(my_cafe, [train, test])
id_test = test.id

('Fix: ', 550)
('Fix: ', 149)


In [22]:
#clean data
equal_index = [601,1896,2791]
test.ix[equal_index, "life_sq"] = test.ix[equal_index, "full_sq"]

kitch_is_build_year = [13117]
train.ix[kitch_is_build_year, "build_year"] = train.ix[kitch_is_build_year, "kitch_sq"]

bad_index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
train.ix[bad_index, "num_room"] = np.NaN

bad_index = [3174, 7313]
test.ix[bad_index, "num_room"] = np.NaN

[train, test] = [clean_sq(train), clean_sq(test)]

train.product_type.value_counts(normalize= True)
test.product_type.value_counts(normalize= True)


train.floor.describe(percentiles= [0.9999])
bad_index = [23584]
train.ix[bad_index, "floor"] = np.NaN
train.material.value_counts()
test.material.value_counts()
train.state.value_counts()
bad_index = train[train.state == 33].index
train.ix[bad_index, "state"] = np.NaN
test.state.value_counts()

2.0    2662
1.0    2266
3.0    1913
4.0     127
Name: state, dtype: int64

In [23]:
# brings error down a lot by removing extreme price per sqm
train.loc[train.full_sq == 0, 'full_sq'] = 50
train = train[train.price_doc/train.full_sq <= 600000]
train = train[train.price_doc/train.full_sq >= 10000]
#mult = .969 train["price_doc"] * mult + 10
train["persqm"] = train["price_doc"]/train.full_sq

In [24]:
[train, test] = [add_dates(train), add_dates(test)]

In [25]:
# Other feature engineering
train, test = my_impute(train), my_impute(test)
train, test = new_features(train), new_features(test)
v=['hospital_beds_raion']
train, test = train.drop(v, axis=1), test.drop(v, axis=1)
test.full_sq = test.full_sq.fillna(1.5*test.life_sq)

In [26]:
from sklearn import model_selection, preprocessing

In [27]:
y_train = train["price_doc"]
x_train = train.drop(["id", "timestamp", "price_doc", "persqm"], axis=1)
x_test = test.drop(["id", "timestamp"], axis=1)

x_train, x_test = labels(x_train), labels(x_test)

In [28]:
v=printna(x_train).index
#x_train, x_test = x_train.drop(v, axis=1), x_test.drop(v, axis=1)
x_train, x_test = x_train.fillna(0), x_test.fillna(0)

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

In [37]:
X=np.array(x_train)
Y=np.array(y_train)
X_test=np.array(x_test)
Norm = MinMaxScaler()
Xt =Norm.fit_transform(X)
Xt_test=Norm.transform(X_test)
Yt = (Y*1.0-min(Y))/(max(Y)-min(Y))

# NN

In [40]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Dropout

Using TensorFlow backend.


In [70]:
def larger_model(dim, weight_reg=0.01, activ_reg=0.01, dp=0.2):
    # create model
    model = Sequential()
    model.add(Dense(dim, input_dim=dim, kernel_initializer='random_normal', activation='relu', 
                    kernel_regularizer=regularizers.l2(weight_reg),activity_regularizer=regularizers.l1(activ_reg)))
    model.add(Dropout(dp))
    model.add(Dense(148, kernel_initializer='random_normal', activation='relu',
                    kernel_regularizer=regularizers.l2(weight_reg),activity_regularizer=regularizers.l1(activ_reg)))
    model.add(Dropout(dp))
    model.add(Dense(20, kernel_initializer='random_normal', activation='relu',
                   kernel_regularizer=regularizers.l2(weight_reg),activity_regularizer=regularizers.l1(activ_reg)))
    model.add(Dropout(dp))
    model.add(Dense(1, kernel_initializer='random_normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [77]:
NN = larger_model(dim=Xt.shape[1], weight_reg=0.01, activ_reg=0.01, dp=0.2)
#NN = load_model('models/drop274r10-0609.h5')

In [80]:
seq=[60]
for i in seq:
    NN.fit(X,Y, batch_size=X.shape[0]/i, epochs=300, verbose=0)
    NN.fit(X,Y, batch_size=X.shape[0]/i, epochs=5, verbose=1)
    print i, r2_score(NN.predict(X),Y)

Epoch 1/5
30403/30403 [==============================] - 0s - loss: 15628479954105.4805     
Epoch 2/5
30403/30403 [==============================] - 0s - loss: 15769416231546.3457     
Epoch 3/5
30403/30403 [==============================] - 0s - loss: 15446850288140.5449     
Epoch 4/5
30403/30403 [==============================] - 0s - loss: 15711142242911.4004     
Epoch 5/5
30403/30403 [==============================] - 0s - loss: 16089292054483.0020     
60 -0.890849788463


In [79]:
NN.predict(X), Y

(array([[ 6731331. ],
        [ 7526640.5],
        [ 6251198. ],
        ..., 
        [ 5731664. ],
        [ 6631263. ],
        [ 5703804.5]], dtype=float32),
 array([ 5850000,  6000000,  5700000, ...,  6970959, 13500000,  5600000]))

In [76]:
print r2_score(NN.predict(Xt),Yt)

-3.5950832562


In [119]:
NN.save('models/drop274r10-0609.h5')

In [124]:
Yt_test = NN.predict(Xt_test)
test['price_doc'] = (max(Y)-min(Y))*Yt_test + min(Y)
#test["price_doc"] =test['persqm']*test["full_sq"]
#out = test[["id","price_doc"]]
#out.to_csv("./nn2.csv", index=False)

In [125]:
test['persqm']

0       5.404131e+07
1       8.171049e+06
2      -4.924844e+05
3       6.487564e+07
4       2.382602e+06
5       2.264605e+06
6       1.608603e+06
7       1.845307e+06
8       9.914066e+06
9       1.040796e+08
10     -7.071814e+05
11      2.178462e+06
12      2.141564e+06
13      8.625219e+05
14      3.829185e+08
15      3.430434e+06
16      1.514366e+06
17      2.136032e+06
18      9.459892e+05
19      9.525724e+06
20      2.119675e+06
21      7.270033e+05
22      8.490811e+05
23      1.606674e+06
24     -2.760079e+06
25     -6.631009e+05
26      2.644038e+05
27     -1.647346e+05
28      1.337572e+06
29      1.649194e+06
            ...     
7632   -4.221885e+05
7633    7.391184e+05
7634    2.349940e+07
7635    1.377595e+06
7636    2.859038e+08
7637    1.478190e+08
7638    2.641069e+08
7639   -5.094874e+05
7640    2.265302e+07
7641    2.705324e+08
7642    1.053888e+06
7643    3.511938e+06
7644    3.348321e+05
7645    2.325257e+08
7646   -4.015542e+04
7647    3.852036e+06
7648    8.566

In [ ]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))